In [ ]:
!pip install pycryptodome
!pip install argon2_cffi
!pip install python-secrets
!apt-get install poppler-utils 
%pip install pdf2image
!pip install opencv-python
!sudo apt install tesseract-ocr
!pip install pytesseract

In [2]:
from Crypto.Cipher import AES
from Crypto.Util import Counter
from Crypto import Random
import binascii
import argon2
import secrets
import os
import cv2
import pytesseract
from pdf2image import convert_from_path,convert_from_bytes

# AES supports multiple key sizes: 16 (AES128), 24 (AES192), or 32 (AES256).
key_bytes = 32

In [3]:
# Takes as input a 32-byte key and an arbitrary-length plaintext and returns a
# pair (iv, ciphtertext). "iv" stands for initialization vector.
def encrypt(key, plaintext):
    assert len(key) == key_bytes

    # Choose a random, 16-byte IV.
    iv = Random.new().read(AES.block_size)

    # Convert the IV to a Python integer.
    iv_int = int(binascii.hexlify(iv), 16) 

    # Create a new Counter object with IV = iv_int.
    ctr = Counter.new(AES.block_size * 8, initial_value=iv_int)

    # Create AES-CTR cipher.
    aes = AES.new(key, AES.MODE_CTR, counter=ctr)

    # Encrypt and return IV and ciphertext.
    ciphertext = aes.encrypt(plaintext)
    return (iv, ciphertext)

In [4]:
# Takes as input a 32-byte key, a 16-byte IV, and a ciphertext, and outputs the
# corresponding plaintext.
def decrypt(key, iv, ciphertext):
    assert len(key) == key_bytes

    # Initialize counter for decryption. iv should be the same as the output of
    # encrypt().
    #iv_int = int(iv.encode('hex'), 16) 
    iv_int = int(binascii.hexlify(iv), 16) 
    ctr = Counter.new(AES.block_size * 8, initial_value=iv_int)

    # Create AES-CTR cipher.
    aes = AES.new(key, AES.MODE_CTR, counter=ctr)

    # Decrypt and return the plaintext.
    plaintext = aes.decrypt(ciphertext)
    return plaintext

In [5]:
def salt_generator():
    #generating salt for better entropy
    #industry recommended standard is to use CSPRNG (Cryptographically Secure Pseudo-Random Number Generator) to produce a salt.

    return secrets.token_bytes(32)

In [6]:
def validation(password):
    #block to generate hash to validate whether the password is correct or not 

    argon2Hasher = argon2.PasswordHasher(time_cost=16, memory_cost=2**15, parallelism=2, hash_len=32, salt_len=16)
    hash = argon2Hasher.hash(password)

    return  hash

In [8]:
def check_password(hash, password):
    #to check whether entered subject(password) by the user is correct or not
    argon2Hasher = argon2.PasswordHasher(time_cost=16, memory_cost=2**15, parallelism=2, hash_len=32, salt_len=16)
       
    try:
        return argon2Hasher.verify(hash, password)
         
    except:
        return False

In [9]:
def key_generator_argon2(password, salt):
    #argon2 is industry specified hashing algorithm. Winner of Password Hashing Competition

    password = password.encode('utf-8')

    hash = argon2.hash_password_raw(time_cost=16, memory_cost=2**15, parallelism=2, hash_len=32,
    password=password, salt=salt, type=argon2.low_level.Type.ID)

    return hash


In [24]:
def encryption(pdf, password):

    text = pdf[0]
    text_to_encrypt = text.encode('utf-8')

    salt = salt_generator()
    key = key_generator_argon2(password, salt)

    hashed_password = validation(password)

    (iv, ciphertext) = encrypt(key, text_to_encrypt)

    return salt, iv, hashed_password, ciphertext

In [10]:
def decryption(salt, iv, ciphertext, password):

    key = key_generator_argon2(password, salt)

    decrypted_data = decrypt(key, iv, ciphertext)

    return decrypted_data

In [18]:
def image_processing(img):
    # get grayscale image
    img1 =  cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # noise removal
    img2 = cv2.medianBlur(img1,5)
    #thresholding
    #return cv2.threshold(img2, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    return cv2.adaptiveThreshold(img2, 255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)

def ocr_to_text(img):
    custom_config = r'--oem 1 --psm 6'
    return pytesseract.image_to_string(img, config=custom_config)

def text_generation(x_data):
  text = []
  for x in x_data:
      a = image_processing(x)
      b = ocr_to_text(a)
      text.append(b)
  return text

def pdf_to_images(pdfs):
      
      pages = convert_from_path(pdfs, 500)
      #pages = convert_from_bytes(pdfs.read(), 500)
      x_data = []
      i = 1
      for page in pages:
          image_name = "Page_" + str(i) + ".jpg"
          page.save(image_name, "JPEG")
          image = cv2.imread(image_name)
          x_data.append(image)
          i = i+1
      text = []
      text = text_generation(x_data)
      i=1
      for page in pages:
          os.remove("Page_" + str(i) + ".jpg")
          i = i+1
      return text

def _main_(download_pdf):
    text = []
    pdfs = download_pdf
    text = pdf_to_images(pdfs)
    return text

In [ ]:
text = _main_("to_present.pdf")
print(text)

In [ ]:
password = "mynameisapples"



(salt, iv, hashed_password, ciphertext) = encryption(text, password)

print(salt)
print(iv)
print(hashed_password)
print(ciphertext)

decrypted_data = decryption(salt, iv, ciphertext, password)

print(decrypted_data)
